In [96]:
import numpy as np
import pandas as pd
import lightning.pytorch as pl
from lightning.pytorch.callbacks import EarlyStopping, LearningRateMonitor
from lightning.pytorch.loggers import TensorBoardLogger
import torch

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

from pytorch_forecasting import TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.data import NaNLabelEncoder
from pytorch_forecasting.metrics import SMAPE

np.random.seed(30)
torch.manual_seed(30)
torch.cuda.manual_seed(30)

In [97]:
df = pd.read_csv("/kaggle/input/ntt-datathon2/clean_dataset.csv")

In [98]:
df.drop(
    columns=["PRECIO", "IMPORTELINEA", "PRODUCTO", "PURCHASING_REGION"], inplace=True
)

In [99]:
df.head()

,FECHAPEDIDO,NUMERO,REFERENCIA,CANTIDADCOMPRA,UNIDADESCONSUMOCONTENIDAS,TIPOCOMPRA,TGL,CODIGO_CLASS,CODIGO_NUM,PURCHASING_HOSPITAL,PURCHASING_DEPARTMENT,YEAR,MONTH,DAYOFMONTH,DAYOFYEAR,time_idx
0,2015-01-02,1312/15,413568,100,5,Compra menor,ALMACENABLE,E,64663,0,2,2015,0.5,0.394356,0.034422,0
1,2015-01-02,1312/15,420680,100,10,Compra menor,ALMACENABLE,E,66071,0,2,2015,0.5,0.394356,0.034422,0
2,2015-01-02,1301/15,1624W,800,400,Compra menor,ALMACENABLE,E,64751,0,2,2015,0.5,0.394356,0.034422,0
3,2015-01-02,1292/15,400403,100,10,Compra menor,ALMACENABLE,B,41691,0,2,2015,0.5,0.394356,0.034422,0
4,2015-01-05,3616/15,420680,160,10,Compra menor,ALMACENABLE,E,66071,10,1,2015,0.5,0.848644,0.085965,1


In [100]:
df.columns

Index(['FECHAPEDIDO', 'NUMERO', 'REFERENCIA', 'CANTIDADCOMPRA',
       'UNIDADESCONSUMOCONTENIDAS', 'TIPOCOMPRA', 'TGL', 'CODIGO_CLASS',
       'CODIGO_NUM', 'PURCHASING_HOSPITAL', 'PURCHASING_DEPARTMENT', 'YEAR',
       'MONTH', 'DAYOFMONTH', 'DAYOFYEAR', 'time_idx'],
      dtype='object')

In [101]:
df["CODIGO_NUM"] = df["CODIGO_NUM"].astype(str)
df["PURCHASING_DEPARTMENT"] = df["PURCHASING_DEPARTMENT"].astype(str)
df["PURCHASING_HOSPITAL"] = df["PURCHASING_HOSPITAL"].astype(str)
df["YEAR"] = df["YEAR"].astype(str)
df["MONTH"] = df["MONTH"].astype(str)

In [102]:
train = df[df["YEAR"] != 2023]
test = df[df["YEAR"] == 2023]

In [103]:
max_prediction_length = 365  # We will predict the entire 2023 year
max_encoder_length = train["time_idx"].nunique()

In [104]:
training = TimeSeriesDataSet(
    train,
    time_idx="time_idx",
    target="CANTIDADCOMPRA",
    group_ids=["CODIGO_NUM", "PURCHASING_DEPARTMENT", "PURCHASING_HOSPITAL"],
    min_encoder_length=max_prediction_length // 4,
    max_encoder_length=max_encoder_length,
    max_prediction_length=max_prediction_length,
    static_categoricals=["CODIGO_NUM", "PURCHASING_DEPARTMENT", "PURCHASING_HOSPITAL"],
    time_varying_known_categoricals=["YEAR", "MONTH"],
    time_varying_known_reals=["time_idx", "DAYOFMONTH", "DAYOFYEAR"],
    time_varying_unknown_categoricals=[],
    time_varying_unknown_reals=["CANTIDADCOMPRA"],
    categorical_encoders={"TIPOCOMPRA": NaNLabelEncoder(add_nan=True)},
    # lags={'CANTIDADCOMPRA': [30, 90, 365]},
    add_relative_time_idx=True,
    add_target_scales=True,
    add_encoder_length=True,
    allow_missing_timesteps=True,
)

validation = TimeSeriesDataSet.from_dataset(
    training, train, predict=True, stop_randomization=True
)

/opt/conda/lib/python3.10/site-packages/pytorch_forecasting/data/timeseries.py:571: UserWarning: Target scales will be only added for continous targets
  warnings.warn("Target scales will be only added for continous targets", UserWarning)
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_arr

In [105]:
batch_size = 64  # set this between 32 to 128
train_dataloader = training.to_dataloader(
    train=True, batch_size=batch_size, num_workers=3
)
val_dataloader = validation.to_dataloader(
    train=False, batch_size=batch_size * 10, num_workers=3
)

In [106]:
PATIENCE = 30
MAX_EPOCHS = 120
LEARNING_RATE = 0.03

In [107]:
early_stop_callback = EarlyStopping(
    monitor="train_loss", min_delta=1e-2, patience=PATIENCE, verbose=False, mode="min"
)
lr_logger = LearningRateMonitor()  # log the learning rate
logger = TensorBoardLogger("lightning_logs")  # logging results to a tensorboard

trainer = pl.Trainer(
    max_epochs=MAX_EPOCHS,
    devices=1,
    accelerator="gpu",
    enable_model_summary=True,
    gradient_clip_val=0.25,
    limit_train_batches=10,  # coment in for training, running valiation every 30 batches
    callbacks=[lr_logger, early_stop_callback],
    logger=logger,
)

tft = TemporalFusionTransformer.from_dataset(
    training,
    learning_rate=LEARNING_RATE,
    lstm_layers=2,
    hidden_size=16,
    attention_head_size=2,
    dropout=0.2,
    hidden_continuous_size=8,
    output_size=1,  # 7 quantiles by default
    loss=SMAPE(),
    log_interval=10,  # uncomment for learning rate finder and otherwise, e.g. to 10 for logging every 10 batches
    reduce_on_plateau_patience=4,
)

tft.to(DEVICE)
print(f"Number of parameters in network: {tft.size()/1e3:.1f}k")

INFO: GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
/opt/conda/lib/python3.10/site-packages/lightning/pytorch/utilities/parsing.py:198: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
/opt/conda/lib/python3.10/site-packages/lightning/pytorch/utilities/parsing.py:198: Attribute 'logging_metrics' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['logging_metrics'])`.


Number of parameters in network: 29.3k


In [110]:
trainer.fit(
    tft,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader,
)

INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
   | Name                               | Type                            | Params
----------------------------------------------------------------------------------------
0  | loss                               | SMAPE                           | 0     
1  | logging_metrics                    | ModuleList                      | 0     
2  | input_embeddings                   | MultiEmbedding                  | 4.0 K 
3  | prescalers                         | ModuleDict                      | 96    
4  | static_variable_selection          | VariableSelectionNetwork        | 944   
5  | encoder_variable_selection         | VariableSelectionNetwork        | 3.4 K 
6  | decoder_variable_selection         | VariableSelectionNetwork        | 2.7 K 
7  | static_context_variable_selection  | GatedResidualNetwork            | 1.1 K 
8  | static_context_initial_hidden_lstm | GatedResidualNetwork            | 1.1 K 
9  | static_context_initia

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=3` in the `DataLoader` to improve performance.
/opt/conda/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=3` in the `DataLoader` to improve performance.
/opt/conda/lib/python3.10/site-packages/lightning/pytorch/loops/fit_loop.py:293: The number of training batches (10) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO: `Trainer.fit` stopped: `max_epochs=120` reached.


In [114]:
best_model_path = trainer.checkpoint_callback.best_model_path
best_tft = TemporalFusionTransformer.load_from_checkpoint(best_model_path)
actuals = torch.cat([y[0] for x, y in iter(val_dataloader)]).to(DEVICE)
predictions = best_tft.predict(val_dataloader, mode="prediction")
raw_predictions = best_tft.predict(val_dataloader, mode="raw", return_x=True)

sm = SMAPE()
print(
    f"Validation median SMAPE loss: {sm.loss(actuals, predictions).mean(axis = 1).median().item()}"
)

INFO: GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Validation median SMAPE loss: 0.3610423505306244
